#### Test llm.py (Llama3 8b, mistral 7b v0.2)

In [27]:
import sys
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/src/')
import llm
import importlib
importlib.reload(sys.modules['llm'])
from llm import LM, OpenAILM

import gc
import torch
from accelerate.utils import release_memory

def flush():
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()

In [28]:

# Specify the custom cache directory
cache_dir = "/data/rech/huiyuche/huggingface"

# empty the memory
flush()
# or
# release_memory()

# llm = LM(
#     model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct",
#     tokenizer_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct",
#     padding_side="left",
#     dtype="bf16",
#     device_map= "auto",
#     use_flash_attention_2=False,
#     access_token=None,
#     cache_dir=cache_dir,
#     accelerator = None
# )
llm = LM(
    model_name_or_path="mistralai/Mistral-7B-Instruct-v0.2",
    tokenizer_name_or_path="mistralai/Mistral-7B-Instruct-v0.2",
    padding_side="left",
    dtype="bf16",
    device_map= "auto",
    attn_implementation="flash_attention_2",
    #use_flash_attention_2=False, (deprecated)
    access_token=None,
    cache_dir=cache_dir,
    accelerator = None
)



/data/rech/huiyuche/envs/trec_ikat/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [02:12<00:00, 44.07s/it]


In [29]:
def bytes_to_giga_bytes(bytes):
    return bytes / 1024 / 1024 / 1024

bytes_to_giga_bytes(torch.cuda.max_memory_allocated())

9.664088726043701

In [30]:
print(type(llm.model))

<class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>


In [31]:
messages = [
    {"role": "user", "content": "what is the capital of France?"},
]
response = llm.hf_llm_generate(
    messages,
    temperature = 1,
    top_p = 0.9,
    max_new_tokens = 256,
    do_sample = True,
    num_beams = 1,
    num_return_sequences = 2
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [32]:
print(response)

(['The capital city of France is Paris. Paris is the most populous city in France and is known for its rich history, art, fashion, gastronomy, and culture. It is one of the most visited cities in the world and is home to many famous landmarks, including the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.', 'The capital city of France is Paris. Paris is one of the most famous cities in the world and is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and the Champs-Élysées. It is also home to many important cultural and artistic institutions, and is a major European business and finance center.'], (tensor([[ -3.3438,  -4.8750, -23.2500,  ...,  -3.1719,  -0.2432,  -1.5156],
        [ -3.3438,  -4.8750, -23.2500,  ...,  -3.1719,  -0.2432,  -1.5156]],
       device='cuda:0'), tensor([[ -3.9062,  -5.0000, -19.8750,  ...,  -2.4531,   0.3223,  -1.7969],
        [ -3.9062,  -5.0000, -19.8750,  ...,  -2.4531,   0.3223,  -1.7969]],
       d

In [33]:
messages = [
    [{"role": "user", "content": "Who are you?"}],
    [{"role": "user", "content": "what is the capital of Germany?"}],
]
outputs = llm.hf_llm_generate_via_pipline(
    messages,
    temperature = 0.1,
    top_p = 1,
    max_new_tokens = 256,
    do_sample = True,
    num_beams = 1,
    num_return_sequences = 2
)

In [34]:
print(outputs)

[[" I am a large language model developed by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a physical presence. I exist solely to process and generate text.", " I am a large language model developed by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a physical presence. I exist solely to process and generate text."], [' The capital city of Germany is Berlin. Berlin is one of the 16 federal states of Germany and is located in the northeastern part of the country. It is the most populous city in Germany and is known for its rich history, diverse culture, and vibrant arts scene. Berlin is home to many famous landmarks, including the Brandenburg Gate, the Reichstag Building, and the Berlin Wall Memorial.', ' The capital city of Germany is Berlin. Berlin is one of the 16 federal states of Germany and is loca

In [35]:
[[alternative["generated_text"][-1]["content"] for alternative in output] for output in outputs ]

TypeError: string indices must be integers, not 'str'

#### get probabilities

### Download LLM

In [ ]:
from accelerate import Accelerator
from accelerate import init_empty_weights
from accelerate import load_checkpoint_and_dispatch

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModel,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    logging
)

from peft import PeftModel, PeftConfig


# Specify the custom cache directory
cache_dir = "/data/rech/huiyuche/huggingface"



# with init_empty_weights():
#     model = AutoModelForCausalLM()

# model = load_checkpoint_and_dispatch(
#     model,
#     checkpoint=cache_dir + "/models--meta-llama--Meta-Llama-3-8B-Instruct",
#     device_map="auto",
# )
# print(model)


# accelerator = Accelerator()
# print(accelerator.device)

#### Mistral 7b v0.2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2", cache_dir=cache_dir
)
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2", cache_dir=cache_dir
)

#### Llama3 8b

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct", cache_dir=cache_dir
)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", cache_dir = cache_dir, device_map="auto")

#### Rankllama

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel, PeftConfig

def get_model(peft_model_name):
    config = PeftConfig.from_pretrained(peft_model_name)
    base_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path, num_labels=1)
    model = PeftModel.from_pretrained(base_model, peft_model_name)
    model = model.merge_and_unload()
    model.eval()
    return model

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
model = get_model('castorini/rankllama-v1-7b-lora-passage')

# Define a query-passage pair
query = "What is llama?"
title = "Llama"
passage = "The llama is a domesticated South American camelid, widely used as a meat and pack animal by Andean cultures since the pre-Columbian era."

# Tokenize the query-passage pair
inputs = tokenizer(f'query: {query}', f'document: {title} {passage}', return_tensors='pt')

# Run the model forward
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    score = logits[0][0]
    print(score)


Loading checkpoint shards: 100%|██████████| 2/2 [02:04<00:00, 62.43s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor(6.1578)


#### Repllama

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from peft import PeftModel, PeftConfig

def get_model(peft_model_name):
    config = PeftConfig.from_pretrained(peft_model_name)
    base_model = AutoModel.from_pretrained(config.base_model_name_or_path, 
                                           cache_dir=cache_dir, 
                                           device_map="auto")
    model = PeftModel.from_pretrained(base_model, peft_model_name)
    model = model.merge_and_unload()
    model.eval()
    return model

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
model = get_model('castorini/repllama-v1-7b-lora-passage')
print(model)
# Define query and passage inputs
# query = "What is llama?"
# title = "Llama"
# passage = "The llama is a domesticated South American camelid, widely used as a meat and pack animal by Andean cultures since the pre-Columbian era."
# query_input = tokenizer(f'query: {query}</s>', return_tensors='pt')
# passage_input = tokenizer(f'passage: {title} {passage}</s>', return_tensors='pt')

# # Run the model forward to compute embeddings and query-passage similarity score
# with torch.no_grad():
#     # compute query embedding
#     query_outputs = model(**query_input)
#     query_embedding = query_outputs.last_hidden_state[0][-1]
#     query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=0)

#     # compute passage embedding
#     passage_outputs = model(**passage_input)
#     passage_embeddings = passage_outputs.last_hidden_state[0][-1]
#     passage_embeddings = torch.nn.functional.normalize(passage_embeddings, p=2, dim=0)

#     # compute similarity score
#     score = torch.dot(query_embedding, passage_embeddings)
#     print(score)


Loading checkpoint shards: 100%|██████████| 2/2 [02:04<00:00, 62.47s/it]


LlamaModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x LlamaDecoderLayer(
      (self_attn): LlamaSdpaAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
        (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
        (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm()
      (post_attention_layernorm): LlamaRMSNorm()
    )
  )
  (norm): LlamaRMSNorm()
)


# Test qrels

In [67]:
import pytrec_eval
qrel_file_path = "/data/rech/huiyuche/TREC_iKAT_2024/data/qrels/toy_qrel.txt"
qrel_file_path = "/data/rech/huiyuche/TREC_iKAT_2024/data/qrels/ikat_23_qrel.txt"
with open(qrel_file_path, 'r') as f_qrel:
    qrel_ndcg = pytrec_eval.parse_qrel(f_qrel)
print(qrel_ndcg["9-1-3"])

{'clueweb22-en0000-00-00251:12': 0, 'clueweb22-en0000-06-17856:4': 2, 'clueweb22-en0000-78-08540:4': 0, 'clueweb22-en0000-84-18471:16': 0, 'clueweb22-en0001-53-12566:3': 3, 'clueweb22-en0002-40-16532:0': 1, 'clueweb22-en0002-44-16189:5': 2, 'clueweb22-en0002-51-09679:0': 0, 'clueweb22-en0003-24-04907:5': 2, 'clueweb22-en0003-65-04965:0': 3, 'clueweb22-en0003-65-04965:1': 4, 'clueweb22-en0003-65-04965:2': 2, 'clueweb22-en0004-12-14927:9': 2, 'clueweb22-en0004-17-01515:2': 3, 'clueweb22-en0004-17-01515:3': 3, 'clueweb22-en0004-35-10824:5': 2, 'clueweb22-en0004-35-10824:6': 2, 'clueweb22-en0004-68-10267:0': 1, 'clueweb22-en0004-68-10267:1': 4, 'clueweb22-en0004-71-01008:7': 2, 'clueweb22-en0004-78-07566:3': 0, 'clueweb22-en0004-90-01678:1': 3, 'clueweb22-en0004-90-01678:2': 2, 'clueweb22-en0004-92-01550:3': 0, 'clueweb22-en0005-03-07986:2': 2, 'clueweb22-en0005-11-08076:0': 0, 'clueweb22-en0005-20-12050:0': 0, 'clueweb22-en0005-70-00162:4': 0, 'clueweb22-en0005-70-00162:5': 0, 'clueweb22-

In [69]:
qrel = {
        qid: {docid: 1 if rel_score > 0 else 0 for docid, rel_score in qrel_ndcg[qid].items()} for qid in qrel_ndcg.keys()
    }
print(qrel["9-1-3"])

{'clueweb22-en0000-00-00251:12': 0, 'clueweb22-en0000-06-17856:4': 1, 'clueweb22-en0000-78-08540:4': 0, 'clueweb22-en0000-84-18471:16': 0, 'clueweb22-en0001-53-12566:3': 1, 'clueweb22-en0002-40-16532:0': 1, 'clueweb22-en0002-44-16189:5': 1, 'clueweb22-en0002-51-09679:0': 0, 'clueweb22-en0003-24-04907:5': 1, 'clueweb22-en0003-65-04965:0': 1, 'clueweb22-en0003-65-04965:1': 1, 'clueweb22-en0003-65-04965:2': 1, 'clueweb22-en0004-12-14927:9': 1, 'clueweb22-en0004-17-01515:2': 1, 'clueweb22-en0004-17-01515:3': 1, 'clueweb22-en0004-35-10824:5': 1, 'clueweb22-en0004-35-10824:6': 1, 'clueweb22-en0004-68-10267:0': 1, 'clueweb22-en0004-68-10267:1': 1, 'clueweb22-en0004-71-01008:7': 1, 'clueweb22-en0004-78-07566:3': 0, 'clueweb22-en0004-90-01678:1': 1, 'clueweb22-en0004-90-01678:2': 1, 'clueweb22-en0004-92-01550:3': 0, 'clueweb22-en0005-03-07986:2': 1, 'clueweb22-en0005-11-08076:0': 0, 'clueweb22-en0005-20-12050:0': 0, 'clueweb22-en0005-70-00162:4': 0, 'clueweb22-en0005-70-00162:5': 0, 'clueweb22-

In [72]:
# read ranking list
ranking_list_path = "/data/rech/huiyuche/TREC_iKAT_2024/results/test/toy_ranking_list.txt"
#ranking_list_path = "/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[oracle_utterance]-S2[none]-g[raw]-[BM25]-[none_4_1_none]-[s2_top50].txt"
with open(ranking_list_path, 'r') as f_run:
    run = pytrec_eval.parse_run(f_run)

print(run)
# pytrec_eval eval
evaluator = pytrec_eval.RelevanceEvaluator(qrel, {"map", "ndcg_cut.5"})
res = evaluator.evaluate(run)
print(res)
evaluator = pytrec_eval.RelevanceEvaluator(qrel_ndcg, {"map", "ndcg_cut.5"})
res = evaluator.evaluate(run)
print(res)

defaultdict(<class 'dict'>, {'9-1-3': {'clueweb22-en0000-00-00251:12': 1020.0, 'clueweb22-en0000-06-17856:4': 1019.0, 'clueweb22-en0000-78-08540:4': 1018.0, 'clueweb22-en0000-84-18471:16': 1017.0, 'clueweb22-en0001-53-12566:3': 1016.0, 'clueweb22-en0002-40-16532:0': 1015.0}})
{'9-1-3': {'map': 0.01206896551724138, 'ndcg_cut_5': 0.34519134224686937}}
{'9-1-3': {'map': 0.01206896551724138, 'ndcg_cut_5': 0.20539694050152013}}


In [ ]:
dict_a = {'a': 1, 'b': 2}
dict_b = {'a': 3, 'b': 5}
dict_c = {'a': 4, 'b': 6}
list_abc = [dict_a, dict_b, dict_c]
sorted_abc = sorted(list_abc[0:2], key=lambda x: x['a'], reverse=True)
sorted_abc[0]['a'] = 100
sorted_abc[1]['a'] = 1000
print(list_abc)

[{'a': 1000, 'b': 2}, {'a': 100, 'b': 5}, {'a': 4, 'b': 6}]


{'9-1-3': {'map': 0.4666666666666666, 'ndcg_cut_5': 0.4776237035032179}}
{'9-1-3': {'map': 0.4666666666666666, 'ndcg_cut_5': 0.4776237035032179}}

'9-1-3': {'map': 1.0, 'ndcg_cut_5': 1.0}}
{'9-1-3': {'map': 1.0, 'ndcg_cut_5': 0.9299817480463239}}

# Test collection


#### jsonl file check, average passage length

In [ ]:
import json
collection_path = "/part/01/Tmp/yuchen/ikat_official_collectioin/ikat_2023_passages_07.jsonl"
# collection_path = "/data/rech/huiyuche/TREC_iKAT_2024/data/collections/ikat_23/cluweb22B_ikat_v2.jsonl" 

num_passage = 0
total_passage_len = 0
with open(collection_path, "r") as f:
    # count the total number of lines in the file
    while True:
        try:
            jsonl_dict = json.loads(f.readline())
            num_passage += 1
            total_passage_len += len(jsonl_dict["contents"].split())
        except:
            break

        # if num_passage > 1000000:
        #     break


average_len = total_passage_len / num_passage
print(f"the average length for {num_passage} passages is: ", average_len)


NameError: name 'num_passsage' is not defined

In [ ]:


print(f"the average length for {num_passage} passages is: ", average_len)

the average length for 7333325 passages is:  201.22463630072306


In [ ]:
collection_path = "/data/rech/huiyuche/TREC_iKAT_2024/data/collections/ikat_23/cluweb22B_ikat_v2.jsonl" 
with open(collection_path, "r") as f:
    # count the total number of lines in the file
    line = f.readline()
    print(line)

In [ ]:
counter = 0
with open("/data/rech/huiyuche/cast23_collection.tsv", "r") as f:
    # count the total number of lines in the file
    line = f.readline()
    while line:
        counter += 1
        line = f.readline()

print("the total number of lines in the file is: ", counter)

In [ ]:
# read the first line

with open ("/data/rech/huiyuche/TREC_iKAT_2024/data/collections/ikat_23/cluweb22B_ikat_v2.jsonl", "r") as f:
    line = f.readline()
    print(line)

{"id": "clueweb22-en0004-50-00485:0", "contents": "RACISM AND THE POLICE: The Shooting of Michael Brown in Ferguson - Sociology Toolbox RACISM AND THE POLICE: The Shooting of Michael Brown in Ferguson Todd Beer on August 18, 2014 *For a look at the patterns that have emerged in the police shootings of unarmed black citizens since the shooting of Michael Brown in Ferguson click here. . . . Systematic racism has been made evident again in the shooting of an unarmed young Black man, Michael Brown, by a police officer in Ferguson, Missouri.  Pulling stories directly from recent news headlines is one way to get students’ attention and demonstrate the abundant relevance of the sociological perspective.  The New York Times has a timeline of the events that serves as a useful starting point (from the mainstream media) to share the events with students that may have not kept up with the story. The community of Ferguson, Missouri (the site of the shooting) has responded with on-going mass protes

In [ ]:
{"id": "clueweb22-en0004-50-00485:0", "contents": "RACISM AND THE POLICE: The Shooting of Michael Brown in Ferguson - Sociology Toolbox RACISM AND THE POLICE: The Shooting of Michael Brown in Ferguson Todd Beer on August 18, 2014 *For a look at the patterns that have emerged in the police shootings of unarmed black citizens since the shooting of Michael Brown in Ferguson click here. . . . Systematic racism has been made evident again in the shooting of an unarmed young Black man, Michael Brown, by a police officer in Ferguson, Missouri.  Pulling stories directly from recent news headlines is one way to get students’ attention and demonstrate the abundant relevance of the sociological perspective.  The New York Times has a timeline of the events that serves as a useful starting point (from the mainstream media) to share the events with students that may have not kept up with the story. The community of Ferguson, Missouri (the site of the shooting) has responded with on-going mass protests. Ferguson cannot be understood in a vacuum. These events are rich with sociological issues –  inequality and poverty, racial profiling, the militarization of the police, protester and police interaction, social media (#Ferguson and hashtag activism) and the “criminalization of Black male youth”. Looking first at the disproportionate levels of poverty and subsequent exclusion from the economy of many Blacks in the US, Brookings, a Democratic leaning think tank, analyzed census tract data of changes in the poverty rates in Ferguson (and the surrounding area) between 2000 and 2008-2012.   They state: “But Ferguson has also been home to dramatic economic changes in recent years."}

#### tsv file check

In [ ]:
line = "clueweb22-en0022-08-02126:19	Personal Requirements: ●	18 years of age or older. ●	Excellent command of the English Language. ●	Computer literate. ●	Quiet working environment. ●	Willing and able to work from home. Technical Requirements: ●	Reliable internet connection at home, with download and upload speeds of at least 1 Mbps. ●	Laptop or Desktop computer. ●	Webcam. ●	Headset/Earphones. Benefits: ●	Work from Home.	https://www.learn4good.com/jobs/language/english/list/teaching/turkey/"

splits= line.strip().split('\t')

print(len(splits))
print(splits)

13
['clueweb22-en0022-08-02126:19', 'Personal Requirements: ●', '18 years of age or older. ●', 'Excellent command of the English Language. ●', 'Computer literate. ●', 'Quiet working environment. ●', 'Willing and able to work from home. Technical Requirements: ●', 'Reliable internet connection at home, with download and upload speeds of at least 1 Mbps. ●', 'Laptop or Desktop computer. ●', 'Webcam. ●', 'Headset/Earphones. Benefits: ●', 'Work from Home.', 'https://www.learn4good.com/jobs/language/english/list/teaching/turkey/']


In [ ]:
line = "clueweb22-en0004-50-00485:0	RACISM AND THE POLICE: The Shooting of Michael Brown in Ferguson - Sociology Toolbox RACISM AND THE POLICE: The Shooting of Michael Brown in Ferguson Todd Beer on August 18, 2014 *For a look at the patterns that have emerged in the police shootings of unarmed black citizens since the shooting of Michael Brown in Ferguson click here. . . . Systematic racism has been made evident again in the shooting of an unarmed young Black man, Michael Brown, by a police officer in Ferguson, Missouri.  Pulling stories directly from recent news headlines is one way to get students’ attention and demonstrate the abundant relevance of the sociological perspective.  The New York Times has a timeline of the events that serves as a useful starting point (from the mainstream media) to share the events with students that may have not kept up with the story. The community of Ferguson, Missouri (the site of the shooting) has responded with on-going mass protests. Ferguson cannot be understood in a vacuum. These events are rich with sociological issues –  inequality and poverty, racial profiling, the militarization of the police, protester and police interaction, social media (#Ferguson and hashtag activism) and the “criminalization of Black male youth”. Looking first at the disproportionate levels of poverty and subsequent exclusion from the economy of many Blacks in the US, Brookings, a Democratic leaning think tank, analyzed census tract data of changes in the poverty rates in Ferguson (and the surrounding area) between 2000 and 2008-2012.   They state: “But Ferguson has also been home to dramatic economic changes in recent years.	https://thesocietypages.org/toolbox/racism-police-ferguson/"
print("the number of tab in a line is", sum([1 for c in line if c == "\t"]) )

the number of tab in a line is 2


# Test general python operation

In [ ]:
a = 2
b = 3
name = 5
def b():
    a()

def a():
    print(name)
    print("abc")


name = "yuchen"
b()

yuchen
abc


In [ ]:
[[{'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': "I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation and answer questions to the best of my knowledge based on my training data.\n\nI'm a large language model, which means I've been trained on a massive dataset of text from various sources, including books, articles, and websites. This training enables me to understand and generate human-like language, allowing me to engage in conversations, answer questions, and even create text based on a given prompt.\n\nI'm constantly learning and improving my responses based on the interactions I have with users like you. So, feel free to ask me anything, and I'll do my best to provide a helpful and accurate response!"}]}]

In [ ]:
[[{'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': " I am a large language model developed by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a physical presence. I exist solely to process and generate text."}]}, {'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': " I am a large language model trained by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate text, and assist with various tasks."}]}]]

In [ ]:
[[{'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': " I am a large language model trained by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate creative text, and assist with various tasks."}]}, {'generated_text': [{'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': " I am a large language model trained by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have feelings, emotions, or a personality. I exist to help answer questions, generate text, and assist with various tasks."}]}], [{'generated_text': [{'role': 'user', 'content': 'what is the capital of Germany?'}, {'role': 'assistant', 'content': ' The capital city of Germany is Berlin. Berlin is one of the 16 federal states of Germany and is located in the northeastern part of the country. It is the most populous city in Germany and is known for its rich history, diverse culture, and vibrant arts scene. Berlin is home to many famous landmarks, including the Brandenburg Gate, the Reichstag Building, and the Berlin Wall Memorial.'}]}, {'generated_text': [{'role': 'user', 'content': 'what is the capital of Germany?'}, {'role': 'assistant', 'content': " The capital city of Germany is Berlin. Berlin is one of the 16 federal states of Germany and is located in the northeastern part of the country. It is the most populous city in Germany and is known for its rich history, diverse culture, and vibrant arts scene. Berlin has been the capital of Germany since the country's reunification in 1990."}]}]]

# Test topics.py


In [1]:
import sys
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/src/')
from topics import (
    Turn, 
    load_turns_from_ikat_topic_files, 
    save_turns_to_json, 
    load_turns_from_json,
    Result,
    Reformulation,
    get_context_by_qid,
    filter_ikat_23_evaluated_turns
)
from dataclasses import asdict
import importlib
importlib.reload(sys.modules['topics'])
from topics import Turn, load_turns_from_ikat_topic_files

In [2]:
list_of_turns = load_turns_from_ikat_topic_files(ikat_topic_file = "/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat23/2023_ikat_test_topics.json")

#### Test get_context_by_qid

In [ ]:
context = get_context_by_qid("9-1-3", list_of_turns)

#### test if dataclasses.asdict works

In [ ]:
lists_of_turns_dict = [asdict(turn) for turn in list_of_turns]
print(lists_of_turns_dict)
# save_to_json
save_turns_to_json(list_of_turns, "/data/rech/huiyuche/TREC_iKAT_2024/test/ikat_2023_test.json")


#### test if load_turns_from_json works

In [ ]:
list_of_turns = load_turns_from_json("/data/rech/huiyuche/TREC_iKAT_2024/test/ikat23/2023_ikat_test_topics.json")

#### test add_reformulation,

In [20]:
turn = list_of_turns[0]
print(turn)
turn.add_reformulation(reformulated_query="emm",reformulation_name="test_name",ptkb_provenance=[])
reformulation = turn.find_reformulation("test_name")
turn.add_result(
    collection = "clueweb12B",
    retrieval_model = "BM25",
    reranker = "rankllama",
    generation_model = "emmm",
    retrieval_query_type = "a",
    reranking_query_type = "b",
    generation_query_type = "c", 
    metrics_dict = {
        "map": 0.5,
        "ndcg_cut_10":0.5
    }
)
print(asdict(turn))

Turn ID: 9-1-1
Title: Finding a diet
Current Utterance: Can you help me find a diet for myself?
Oracle Utterance: Can you help me find a diet for myself considering that I'm vegetarian, allergic to soybeans, lactose intolerant, can't exercise too much, and should drink water regularly?
Number of reformulations: 1
Number of results: 1

{'turn_id': '9-1-1', 'conversation_id': '9-1', 'title': 'Finding a diet', 'current_utterance': 'Can you help me find a diet for myself?', 'current_response': 'Sure, these diets fit your condition and preference: 1) Vegan Mediterranean combines the principles of the Mediterranean diet with veganism and emphasizes whole grains, vegetables, beans, nuts, seeds, herbs, and spices. It excludes animal foods while incorporating plant-based alternatives, offering a sustainable and compassionate approach. (2) Vegan Keto: is a high-fat, low-carbohydrate, and moderate-protein diet which is also exclusively plant-based. It suits individuals following a vegan lifestyle

In [ ]:
turn_dict = asdict(turn)
print(turn_dict)

{'turn_id': '9-1-1', 'conversation_id': '9-1', 'title': 'Finding a diet', 'current_utterance': 'Can you help me find a diet for myself?', 'current_response': 'Sure, these diets fit your condition and preference: 1) Vegan Mediterranean combines the principles of the Mediterranean diet with veganism and emphasizes whole grains, vegetables, beans, nuts, seeds, herbs, and spices. It excludes animal foods while incorporating plant-based alternatives, offering a sustainable and compassionate approach. (2) Vegan Keto: is a high-fat, low-carbohydrate, and moderate-protein diet which is also exclusively plant-based. It suits individuals following a vegan lifestyle who wish to adopt a ketogenic eating plan. (3) Eco-Atkins is a vegetarian variation of the Atkins diet which focuses on whole grains, vegetables, tofu, legumes, nuts, and seeds while minimizing animal products. By reducing carbohydrate intake and incorporating plant-based protein sources, it promotes sustainability and healthier eatin


#### test from_dict

In [ ]:
new_turn = Turn()
new_turn.from_dict(turn_dict)
print(new_turn.reformulations)

[Reformulation(reformulation_name='test_name', reformulated_query='emm')]


#### change turn object format (or Result, Reformulation object format)

In [8]:
import sys
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/src/')
import importlib
import topics
importlib.reload(sys.modules['topics'])
from topics import Turn, load_turns_from_ikat_topic_files, save_turns_to_json, load_turns_from_json,Result,Reformulation
from dataclasses import asdict
import json

# current version
ikat_topic_file = "/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat23/ikat_2023_test.json"


# modified version
#where_to_save = "/data/rech/huiyuche/TREC_iKAT_2024/test/ikat_2023_test.json"
where_to_save = "/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat23/ikat_2023_test.json"


In [ ]:
list_of_turns = load_turns_from_json(ikat_topic_file)
turn_dict_list = save_turns_to_json(list_of_turns, where_to_save)
print(json.dumps(turn_dict_list[2], indent=4))

#### rectify minor errors in the topic file

In [9]:
list_of_turns = load_turns_from_json(ikat_topic_file)
for turn in list_of_turns:
    for result in turn.results:
       if type(result.metrics) == type([]):
           result.metrics = result.metrics[0] 

turn_dict_list = save_turns_to_json(list_of_turns, where_to_save)

### Test IKAT format output

In [ ]:
import json
ikat_output_path = "/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ikat_format_output/none.json"
with open(ikat_output_path, "r") as f:
    ikat_output = json.load(f)
print(ikat_output["turns"][0])


{'turn_id': '10-2_13', 'responses': [{'rank': 1, 'text': 'pseudo response 1', 'ptkb_provenance': [], 'passage_provenance': [{'id': 'clueweb22-en0013-24-12486:0', 'score': 16.136999130249023, 'used': True}, {'id': 'clueweb22-en0044-24-12435:0', 'score': 16.068199157714844, 'used': True}, {'id': 'clueweb22-en0012-73-01681:8', 'score': 15.756500244140625, 'used': True}, {'id': 'clueweb22-en0041-03-09116:0', 'score': 15.668299674987793, 'used': False}, {'id': 'clueweb22-en0025-44-01168:0', 'score': 15.497599601745605, 'used': False}]}]}


### Case study: Why human rewrite is better?

#### compare 2 rewrites

In [2]:
list_of_turns = load_turns_from_json("/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat23/ikat_2023_test.json")
list_of_turns = filter_ikat_23_evaluated_turns(list_of_turns)

In [4]:
list_of_dominant_oracle = {}
a_name = "oracle_utterance"
b_name = "rar_rw"
metric_name = "recall_50"
stage_name = "retrieval"

red_square = "\U0001F7E5"
green_square = "\U0001F7E9"

output_path = f"/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/analysis/[{a_name}]vs[{b_name}]on_[{metric_name}]_at_[{stage_name}].txt"


In [ ]:
for turn in list_of_turns:
    better, a_performance, b_performance =\
        turn.rewrite_a_is_better_than_b_on_at_stage(
            a_name=a_name,
            b_name=b_name,
            metric_name = metric_name,
            stage = stage_name
            )
    if better:
        color = green_square
    else:
        color = red_square

    with open(output_path, "a") as f:
        f.write("##########################################\n")
        f.write(f"The turn id is: {turn.turn_id} {color}\n")
        f.write("############################\n")
        f.write(f"The scores for metric [{metric_name}] are:\n {a_name}: {a_performance:.4f}\n {b_name}: {b_performance:.4f}\n")
        f.write("############################\n")
        f.write(f"the original query is:\n{turn.current_utterance}\n")
        f.write("############################\n")
        f.write(f"the [{a_name}] is:\n{turn.query_type_2_query(a_name,20,0.5)}\n")
        f.write("############################\n")
        f.write(f"the [{b_name}] is:\n{turn.query_type_2_query(b_name,20,0.5)}\n")
        f.write("***************************\n")
        f.write("Reason:\n")
        f.write("***************************\n")

#### function: load document by id

In [7]:

import pyserini
from pyserini.setup import configure_classpath
#configure_classpath("/data/rech/huiyuche/envs/trec_ikat/lib/python3.12/site-packages/pyserini/resources/jars")
from jnius import autoclass

# get collection reader
JIndexReaderUtils = autoclass('io.anserini.index.IndexReaderUtils')
reader = JIndexReaderUtils.getReader("/part/01/Tmp/yuchen/indexes/clueweb22b_ikat23_fengran_sparse_index_2/")

def load_document_by_id(doc_id, reader):

    doc_content = JIndexReaderUtils.documentRaw(reader, doc_id)
    return doc_content


#### get all evaluated docs for a query

In [47]:
from pyserini.setup import configure_classpath
#configure_classpath("/data/rech/huiyuche/envs/trec_ikat/lib/python3.12/site-packages/pyserini/resources/jars")
from jnius import autoclass
import pytrec_eval


qid = "10-3-5"
qrel_file_path = "/data/rech/huiyuche/TREC_iKAT_2024/data/qrels/ikat_23_qrel.txt"
output_path = f"/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/analysis/all_relevant_docs_for[{qid}].txt"


with open(qrel_file_path, 'r') as f_qrel:
    qrel = pytrec_eval.parse_qrel(f_qrel)

# get all relevant doc ids for the query
doc_id_list = [doc_id for doc_id in qrel[qid].keys() ]

# sort the doc_id_list by the score
doc_id_list = sorted(doc_id_list, key=lambda doc_id: qrel[qid][doc_id], reverse=True)

with open(output_path, "w") as f:
    f.write("There are total of {} relevant documents for the query {}\n".format(len(doc_id_list), qid))

with open(output_path, "a") as f:
    for doc_id in doc_id_list:
        # Fetch raw document contents by id:
        doc_content = load_document_by_id(doc_id, reader)
        
        f.write("##########################################\n")
        f.write(f"The relevance score is: {qrel[qid][doc_id]}\n")
        f.write("##############################\n")
        f.write(f"{doc_content}\n")



#### single query searcher evaluator

In [11]:
from pyserini.search.lucene import LuceneSearcher
import pytrec_eval

retrieval_query_list = [
    #"DASH diet healthy diet.",
    "What about the DASH diet? I heard it is a healthy diet."
    #Dietary Approaches to Stop Hypertension",
    #"I prefer a natural diet, not a pill-based diet. Which of the aforementioned ones is natural? Among the diets mentioned, the Vegan Mediterranean and Eco-Atkins diets are more natural options as they focus on whole plant-based foods like vegetables, fruits, grains, nuts, and seeds. These diets do not rely on pills or supplements, making them a more natural choice for your dietary preferences.",
    #"Can fish be included in any of the diets you mentioned? Fish is not included in the Vegan Mediterranean, Vegan Keto, Eco-Atkins, or Ornish Diets as they are all plant-based diets that exclude animal products, including fish. If you are looking to include fish in your diet, you may need to consider other dietary options that allow for the consumption of seafood."
    #"Vegan Mediterranean, Vegan Keto, Eco-Atkins, Ornish Diets, fish, fish, fish, fish,animal, plant, included, included, included",
    #"how to select choose a car",
    #"Italian Italian italian recipe dishes,  dish, dish, ingredient, tomato "
    #这个还可以吧.."italian dish, italian food, italian cuisine, italian,italian, recipe, chicken, chicken, pasta, pasta tomato, tomato mozzarella cheese, ingredient "
    #"cars, car, best, midsize, advice, checklist, need, buy, buying, friendly, look, different, motors, news, right, choosing, easiest, thing, before, taking."
    #"Which of the lasagne, gnocchi ai quattro formaggi, melanzane di parmigiana, arancini, minestrone, dolmade, grilled meat, and mussake suits the preferences of me and my family?"#Vanmoof, lamb,"# chicken tuna, allergic eggplants turkey, lactose , salmon, mediterranean, Netherlands"
    #"now what can I do to lose some weight?"
    #"Which of the Christmas ornaments, sauerkraut juice, feuerzangenbowle set, dirndl and/or lederhosen, ampelmannchen, beer steins, chocolate, German gummy bears, cuckoo clocks, German kitchen tools, and utensils, wooden toys for children, seasonal tablecloths, egg liqueur, German tea, German wine from Saxony, German herbs and spices, and Birkenstock do you think my aunt would like more, as a highly skilled chef?"
    #"Do you know any stores for WMF in Berlin"
    #"Thanks. What about some traditional and history souvenir from Germany for my uncle?"


]
qid_list_string = [
    "9-1-3",
]

query_dict = {
    qid: query for qid, query in zip(qid_list_string, retrieval_query_list)
}

searcher = LuceneSearcher("/part/01/Tmp/yuchen/indexes/clueweb22b_ikat23_fengran_sparse_index_2/")
searcher.set_bm25(0.9, 0.4)
hits = searcher.batch_search(retrieval_query_list, qid_list_string, k = 1000, threads = 40)

In [12]:
import json
# form the run dictionary required by pytrec_eval
run = {qid: {doc.docid: doc.score for doc in docs} for qid, docs in hits.items()}

# get qrel
qrel_file_path = "/data/rech/huiyuche/TREC_iKAT_2024/data/qrels/ikat_23_qrel.txt"
with open(qrel_file_path, 'r') as f_qrel:
    qrel = pytrec_eval.parse_qrel(f_qrel)

# pytrec_eval eval
evaluator = pytrec_eval.RelevanceEvaluator(qrel, {"map", "ndcg", "ndcg_cut.1", "ndcg_cut.3", "ndcg_cut.5", "ndcg_cut.10", "P.1", "P.3", "P.5", "P.10", "P.20", "recall.5", "recall.20", "recall.50", "recall.100", "recall.1000", "recip_rank"})
res = evaluator.evaluate(run)
print(json.dumps(res, indent=4))

{
    "9-1-3": {
        "map": 0.21365648820982636,
        "recip_rank": 1.0,
        "P_1": 1.0,
        "P_3": 1.0,
        "P_5": 1.0,
        "P_10": 1.0,
        "P_20": 0.7,
        "recall_5": 0.04310344827586207,
        "recall_20": 0.1206896551724138,
        "recall_50": 0.1810344827586207,
        "recall_100": 0.25862068965517243,
        "recall_1000": 0.5862068965517241,
        "ndcg": 0.46847209170065474,
        "ndcg_cut_1": 0.5,
        "ndcg_cut_3": 0.5432992037642228,
        "ndcg_cut_5": 0.5312934851347721,
        "ndcg_cut_10": 0.5293856728621718
    }
}


In [9]:
doc_id_to_test = [
    "clueweb22-en0044-81-02940:3",
    "clueweb22-en0034-59-19965:3",
    "clueweb22-en0022-85-11301:3",
    "clueweb22-en0044-46-01525:0",
    "clueweb22-en0004-43-04866:4",
    "clueweb22-en0040-47-04088:7"
]
for doc_id in doc_id_to_test:
    print(doc_id in [doc.docid for doc in hits["10-3-6"]])

False
False
True
False
False
False


#### yield top n search results

In [10]:
top_n = 20
qid = "9-1-3"
output_path = f"/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/analysis/top_{top_n}_searched_docs_for_query_{qid}.txt"
qrel_file_path = "/data/rech/huiyuche/TREC_iKAT_2024/data/qrels/ikat_23_qrel.txt"


query = query_dict[qid]

with open(qrel_file_path, 'r') as f_qrel:
    qrel = pytrec_eval.parse_qrel(f_qrel)

print(qrel)
# get all evaluated doc ids for the query
evaluated_doc_id_list = [doc_id for doc_id in qrel[qid].keys() ]

# number of top n documents that are evaluated
top_n_evaluated_doc_id_list = [doc.docid for doc in hits[qid][:top_n] if doc.docid in evaluated_doc_id_list]
num_already_evaluated = len(top_n_evaluated_doc_id_list)

with open(output_path, "w") as f:
    f.write(f"The top {top_n} searched documents for the query {qid}: [{query}], where {num_already_evaluated} are already evaluated\n")



with open(output_path, "a") as f:
    for doc in hits[qid][:top_n]:
        # Fetch raw document contents by id:
        doc_content = load_document_by_id(doc.docid, reader)

        evaluated = False
        if doc.docid in evaluated_doc_id_list:
            evaluated = True
        
        if evaluated:
            evaluated_score = qrel[qid][doc.docid]
        else:
            evaluated_score = "not evaluated"
            
        
        f.write("##########################################\n")
        f.write(f"The relevance score is: {doc.score}, The evaluated score is: {evaluated_score}\n")
        f.write("##############################\n")
        f.write(f"{doc_content}\n")

defaultdict(<class 'dict'>, {'9-1-3': {'clueweb22-en0000-00-00251:12': 0, 'clueweb22-en0000-06-17856:4': 2, 'clueweb22-en0000-78-08540:4': 0, 'clueweb22-en0000-84-18471:16': 0, 'clueweb22-en0001-53-12566:3': 3, 'clueweb22-en0002-40-16532:0': 1, 'clueweb22-en0002-44-16189:5': 2, 'clueweb22-en0002-51-09679:0': 0, 'clueweb22-en0003-24-04907:5': 2, 'clueweb22-en0003-65-04965:0': 3, 'clueweb22-en0003-65-04965:1': 4, 'clueweb22-en0003-65-04965:2': 2, 'clueweb22-en0004-12-14927:9': 2, 'clueweb22-en0004-17-01515:2': 3, 'clueweb22-en0004-17-01515:3': 3, 'clueweb22-en0004-35-10824:5': 2, 'clueweb22-en0004-35-10824:6': 2, 'clueweb22-en0004-68-10267:0': 1, 'clueweb22-en0004-68-10267:1': 4, 'clueweb22-en0004-71-01008:7': 2, 'clueweb22-en0004-78-07566:3': 0, 'clueweb22-en0004-90-01678:1': 3, 'clueweb22-en0004-90-01678:2': 2, 'clueweb22-en0004-92-01550:3': 0, 'clueweb22-en0005-03-07986:2': 2, 'clueweb22-en0005-11-08076:0': 0, 'clueweb22-en0005-20-12050:0': 0, 'clueweb22-en0005-70-00162:4': 0, 'cluewe

# Test rerank

In [ ]:
import sys
import importlib
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/src/')
import rerank
from rank_gpt import run_retriever, sliding_windows
importlib.reload(sys.modules['rerank'])

from rerank import (
    load_rankllama, 
    rerank_rankllama,
    hits_2_rankgpt_list
    )

cache_dir = "/data/rech/huiyuche/huggingface"


query = "what is the capital of China?"

passages = [
    "The capital of China is Beijing.",
    "Beijing. The modern day capital of China is Beijing (literally 'Northern Capital'), which first served as China's capital city in 1261, when the Mongol ruler Kublai Khan established his seat of power in the area centered around what is today Beihai Park.",
    "The capital of France is Paris.",
    "Llama The llama is a domesticated South American camelid, widely used as a meat and pack animal by Andean cultures since the pre-Columbian era."
]



In [ ]:
tokenizer, model = load_rankllama(cache_dir)

In [ ]:
scores = rerank_rankllama(query, passages, tokenizer, model)
print(scores) 

[tensor(3.4062, dtype=torch.bfloat16), tensor(4.4688, dtype=torch.bfloat16), tensor(-3.9062, dtype=torch.bfloat16), tensor(-9.8125, dtype=torch.bfloat16)]


In [ ]:
print(type(model))
print(type(tokenizer))

<class 'transformers.models.llama.modeling_llama.LlamaForSequenceClassification'>
<class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>


#### rankGPT

In [ ]:
import os
os.system("source ~/.bashrc")
openai_key = os.getenv('openai_key')

#### data structure of hits

In [ ]:
from pyserini.search.lucene import LuceneSearcher
LuceneSearcher.list_prebuilt_indexes()

In [ ]:
from pyserini.search.lucene import LuceneSearcher

retrieval_query_list = [
    "what is the capital of France?",
    "what is the capital of Germany?",
    "what is the capital of Italy?"
]
qid_list_string = [
    "id1",
    "id2",
    "id3"
]

query_dict = {
    qid: query for qid, query in zip(qid_list_string, retrieval_query_list)
}

searcher = LuceneSearcher("/part/01/Tmp/yuchen/indexes/clueweb22b_ikat23_fengran_sparse_index_2/")
searcher.set_bm25(0.9, 0.4)
hits = searcher.batch_search(retrieval_query_list, qid_list_string, k = 10, threads = 40)

/data/rech/huiyuche/envs/trec_ikat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Jul 09, 2024 5:51:28 A.M. org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


In [ ]:
import numpy as np
liste = np.array([1,3,2,4,5], dtype=np.float32)
index = np.argsort(-liste)
print(index)

[4 3 1 2 0]


In [ ]:
hits["id1"][0].rank = 0
print(hits["id1"][0].rank)

0


In [ ]:
import json
rank_gpt_list, new_hits_dict = hits_2_rankgpt_list(searcher, query_dict, hits)
for i in range(3):
    rank_gpt_list[i]["hits"].append(
        {
            "content" : "Paris, the capital of France, was originally a small tribal settlement in 3rd century BC. It flourished under the Romans, becoming a prosperous city. In the Middle Ages, it became the capital of a major European kingdom and a center of learning. Paris played a central role in the French Revolution and was transformed into a modern city in the 19th century by Napoleon III. Today, it's a leading global city known for art, fashion, and culture",
            "qid":"id1",
            "docid": "pseudo-id",
            "rank": 10,
            "score":0.1
        }
    )
print(json.dumps(rank_gpt_list[0:1], indent=4))

[
    {
        "query": "what is the capital of France?",
        "hits": [
            {
                "content": "What is the Capital of France? | Mappr Capital Cities What is the Capital of France? What is the Capital of France? Paris is the capital city of France and the center of France. It is built on the Seine River, in the middle of the Paris Basin. Where is Paris? Known for its monuments, artistic and cultural life all over the world, Paris is also one of the major economic and political centers along with being an important city in world history and is one of the transit points of international transport. Paris, a fashion and luxury world capital, is also known as \u201cCity of Lights\u201d. When did Paris Become the Capital? The most important archaeological finds are the remains of the oldest permanent human settlement in the Paris region, which was discovered in 1991 in the 12th region.",
                "qid": "id1",
                "docid": "clueweb22-en0017-32-04653:

In [ ]:
from tqdm import tqdm
new_rank_gpt_list = []
rank_end = 11
for item in tqdm(rank_gpt_list):
    new_item = sliding_windows(item, rank_start=0, rank_end=rank_end, window_size=5, step=2, model_name='gpt-3.5-turbo', api_key=openai_key)
    new_rank_gpt_list.append(new_item)


100%|██████████| 3/3 [00:10<00:00,  3.43s/it]


In [ ]:
print(json.dumps(new_rank_gpt_list[0:1], indent=4))

[
    {
        "query": "what is the capital of France?",
        "hits": [
            {
                "content": "What is the Capital of France? | Mappr Capital Cities What is the Capital of France? What is the Capital of France? Paris is the capital city of France and the center of France. It is built on the Seine River, in the middle of the Paris Basin. Where is Paris? Known for its monuments, artistic and cultural life all over the world, Paris is also one of the major economic and political centers along with being an important city in world history and is one of the transit points of international transport. Paris, a fashion and luxury world capital, is also known as \u201cCity of Lights\u201d. When did Paris Become the Capital? The most important archaeological finds are the remains of the oldest permanent human settlement in the Paris region, which was discovered in 1991 in the 12th region.",
                "qid": "id1",
                "docid": "clueweb22-en0017-32-04653:

#### monoT5

In [ ]:
import sys
cache_dir = "/data/rech/huiyuche/huggingface"
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/src/')
import importlib
import rerank
import llm
importlib.reload(sys.modules['llm'])
importlib.reload(sys.modules['rerank'])
from rerank import (
    load_t5_DP,
    rerank_t5_DP
)


2024-07-10 05:10:47.229103: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-10 05:10:49.215882: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possib

In [ ]:
tokenizer, model, decoder_stard_id, targeted_ids = load_t5_DP(
    cache_dir = cache_dir,
    model_name = "castorini/monot5-base-msmarco",
)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Ready for predict()


In [ ]:
print(tokenizer.encode("what is the capital of France?"))

[125, 19, 8, 1784, 13, 1410, 58, 1]


In [ ]:
from pyserini.search.lucene import LuceneSearcher

retrieval_query_list = [
    "what is the capital of France?",
    "what is the capital of Germany?",
    "what is the capital of Italy?"
]
qid_list_string = [
    "id1",
    "id2",
    "id3"
]

query_dict = {
    qid: query for qid, query in zip(qid_list_string, retrieval_query_list)
}

searcher = LuceneSearcher("/part/01/Tmp/yuchen/indexes/clueweb22b_ikat23_fengran_sparse_index_2/")
#searcher = LuceneSearcher("/data/rech/huiyuche/TREC_iKAT_2024/data/indexes/clueweb22b_ikat23_fengran_sparse_index_2")
searcher.set_bm25(0.9, 0.4)
hits = searcher.batch_search(retrieval_query_list, qid_list_string, k = 1000, threads = 40)

In [ ]:
 from tqdm import tqdm
 import json
 for qid, hit in tqdm(hits.items(), total=len(hits), desc="Reranking"):

    reranking_query = query_dict[qid]
    reranked_scores = rerank_t5_DP(
        reranking_query,
        [json.loads(searcher.doc(doc_object.docid).raw())["contents"] for doc_object in hit],
        tokenizer,
        model,
        decoder_stard_id,
        targeted_ids,
    )

    print(reranked_scores)

Reranking:   0%|          | 0/3 [00:00<?, ?it/s]/data/rech/huiyuche/envs/trec_ikat/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Reranking:  33%|███▎      | 1/3 [00:16<00:32, 16.32s/it]

[0.9456437230110168, 0.996783971786499, 0.9666732549667358, 6.670828042842913e-06, 0.0817103460431099, 0.7397226095199585, 0.00876297801733017, 0.0719098299741745, 2.736906481004553e-06, 4.171901673544198e-05, 0.03734677657485008, 0.9959157109260559, 7.1600134106120095e-06, 0.0015542390756309032, 0.9418118596076965, 7.144066785258474e-06, 0.00045250411494635046, 0.00036305320099927485, 2.427258550596889e-05, 2.3689308363827877e-05, 0.9712644219398499, 0.24110576510429382, 0.0002644523046910763, 0.5568421483039856, 0.00040787336183711886, 0.9873769879341125, 0.03674014285206795, 0.8245005011558533, 0.00019249253091402352, 0.8373112678527832, 0.5780085921287537, 7.149000884965062e-05, 0.9149550199508667, 0.00017964046855922788, 0.9679471850395203, 0.9127674102783203, 0.00016537107876501977, 0.0011061328696087003, 1.9488272300804965e-05, 5.129402325110277e-06, 0.9169191122055054, 0.05827391520142555, 1.4319890624392428e-06, 0.001757705700583756, 8.745069862925448e-06, 0.001048920792527496

Reranking:  67%|██████▋   | 2/3 [00:30<00:14, 14.89s/it]

[0.9953970313072205, 2.01152920453751e-06, 0.7174405455589294, 0.8589284420013428, 6.07621950621251e-06, 0.0001394230785081163, 1.5411380445584655e-05, 0.16442130506038666, 0.006078634411096573, 0.13731539249420166, 0.8877601623535156, 9.802843851502985e-05, 1.456777044950286e-05, 0.0008441982208751142, 0.00040752513450570405, 0.1008419543504715, 1.0655906407919247e-05, 0.002407826716080308, 0.0008673791890032589, 0.8346805572509766, 0.0061761499382555485, 0.4766225814819336, 0.00020182768639642745, 0.819810688495636, 0.3919393718242645, 0.06538358330726624, 0.00026937693473882973, 4.998672466172138e-06, 0.0003038840659428388, 6.597803803742863e-06, 0.01524920854717493, 0.00011770183482440189, 2.0283585399738513e-05, 2.350113027205225e-05, 0.024353962391614914, 1.7835785683928407e-06, 7.114362233551219e-06, 0.4937599301338196, 1.3999332622915972e-05, 0.010468983091413975, 1.4748639841855038e-05, 2.040050458163023e-05, 2.4147428121068515e-05, 2.1775629647891037e-05, 0.000378146243747323

Reranking: 100%|██████████| 3/3 [00:44<00:00, 14.81s/it]

[0.017968302592635155, 0.9974547028541565, 0.01277640275657177, 0.0001642050192458555, 0.9243150353431702, 0.5052817463874817, 0.9671682119369507, 0.00034803044400177896, 5.477433660416864e-05, 5.4697349696652964e-05, 0.694668173789978, 2.2093896404840052e-05, 6.178530838951701e-06, 9.633455920265988e-05, 2.332322401343845e-05, 1.3590860362455714e-05, 0.000193543586647138, 0.7538589835166931, 0.6507065892219543, 0.8190565705299377, 0.5761234164237976, 0.4250348210334778, 5.779129787697457e-05, 1.1080785043304786e-05, 0.0002873626654036343, 0.0001381249603582546, 0.3830854892730713, 0.0001821334008127451, 0.0031503415666520596, 0.000164004901307635, 0.9741204380989075, 3.930743332603015e-06, 0.016553279012441635, 2.3085483462637058e-06, 7.739575812593102e-05, 0.9647181034088135, 0.9194992184638977, 0.9082461595535278, 0.0010069620329886675, 4.751959568238817e-05, 0.0005041667609475553, 0.0006544327479787171, 0.03802913427352905, 9.870432222669479e-06, 5.651775609294418e-06, 0.0006206319

: 

#### test DDP

In [ ]:
import os
import sys
import importlib
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/test/')
import test_ddp
importlib.reload(sys.modules['test_ddp'])
from test_ddp import example, aha 

os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"
# Run the aha function
output = aha()

<class 'multiprocessing.managers.ListProxy'>
i =  0


Model on device:  cuda:3
Model on device:  cuda:1
Model on device:  cuda:2
Model on device:  cuda:0
i =  1
Model on device:  cuda:1
Model on device:  cuda:3
Model on device:  cuda:2
Model on device:  cuda:0
i =  2
Model on device:  cuda:2
Model on device:  cuda:1
Model on device:  cuda:3
Model on device:  cuda:0
i =  3
Model on device:  cuda:3
Model on device:  cuda:2
Model on device:  cuda:1
Model on device:  cuda:0
i =  4
Model on device:  cuda:3
Model on device:  cuda:2
Model on device:  cuda:1
Model on device:  cuda:0


#### test monot5 ddp

In [ ]:

import sys
cache_dir = "/data/rech/huiyuche/huggingface"
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/src/')
import importlib
import rerank
import llm
importlib.reload(sys.modules['llm'])
importlib.reload(sys.modules['rerank'])
from rerank import (
    load_t5_DDP,
    rerank_t5_DDP
)

In [ ]:
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer,
    T5ForConditionalGeneration,
    PreTrainedTokenizer
    )

tokenizer, model, decoder_stard_id, targeted_ids = load_t5_DDP(
    cache_dir = cache_dir,
    model_name = "castorini/monot5-base-msmarco",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Ready for predict()


In [ ]:

from pyserini.search.lucene import LuceneSearcher

retrieval_query_list = [
    "what is the capital of France?",
    "what is the capital of Germany?",
    "what is the capital of Italy?"
]
qid_list_string = [
    "id1",
    "id2",
    "id3"
]

query_dict = {
    qid: query for qid, query in zip(qid_list_string, retrieval_query_list)
}

searcher = LuceneSearcher("/part/01/Tmp/yuchen/indexes/clueweb22b_ikat23_fengran_sparse_index_2/")
#searcher = LuceneSearcher("/data/rech/huiyuche/TREC_iKAT_2024/data/indexes/clueweb22b_ikat23_fengran_sparse_index_2")
searcher.set_bm25(0.9, 0.4)
hits = searcher.batch_search(retrieval_query_list, qid_list_string, k = 1000, threads = 40)

/data/rech/huiyuche/envs/trec_ikat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Jul 08, 2024 8:53:59 A.M. org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


In [ ]:
id1_first_5_ids_1000 = [doc_object.docid for doc_object in hits["id1"][:5]]
id1_first_5_scores_1000 = [doc_object.score for doc_object in hits["id1"][:5]]
print(id1_first_5_ids_1000)
print(id1_first_5_scores_1000)

['clueweb22-en0017-32-04653:0', 'clueweb22-en0016-24-18543:0', 'clueweb22-en0021-14-14889:63', 'clueweb22-en0037-88-06710:10', 'clueweb22-en0001-30-00978:0']
[9.22029972076416, 8.990500450134277, 8.907099723815918, 8.903499603271484, 8.807000160217285]


In [ ]:

hits = searcher.batch_search(retrieval_query_list, qid_list_string, k = 50, threads = 40)

In [ ]:
id1_first_5_ids_50 = [doc_object.docid for doc_object in hits["id1"][:5]]
id1_first_5_scores_50 = [doc_object.score for doc_object in hits["id1"][:5]]
print(id1_first_5_ids_50)
print(id1_first_5_scores_50)

['clueweb22-en0017-32-04653:0', 'clueweb22-en0016-24-18543:0', 'clueweb22-en0021-14-14889:63', 'clueweb22-en0037-88-06710:10', 'clueweb22-en0001-30-00978:0']
[9.22029972076416, 8.990500450134277, 8.907099723815918, 8.903499603271484, 8.807000160217285]


In [ ]:

from tqdm import tqdm
import json
import torch

# to add to evaluation.py
import torch.multiprocessing as mp
from multiprocessing import Manager

# DDP related configuration
world_size = torch.cuda.device_count()
print("world_size", world_size)

manager = Manager()
output_list = manager.list()


for qid, hit in tqdm(hits.items(), total=len(hits), desc="Reranking"):

    reranking_query = query_dict[qid]

    mp.spawn(rerank_t5_DDP,
            args=(
                world_size, 
                reranking_query,
                [json.loads(searcher.doc(doc_object.docid).raw())["contents"] for doc_object in hit],
                tokenizer,
                decoder_stard_id,
                targeted_ids,
                output_list
                ),
            nprocs=world_size,
            join=True)


    reranked_score = list(output_list)
    print(len(reranked_score))
    print(reranked_score[0])

world_size 4


Reranking:   0%|          | 0/3 [00:00<?, ?it/s]2024-07-04 02:27:27.199244: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-04 02:27:29.468771: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-07-04 02:27:38.162606: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-04 02:27:39.956140: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You are using the default legacy behaviour of the <class 'transformers.models.t5

Ready for predict()


2024-07-04 02:27:47.561211: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-04 02:27:49.444943: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tu

Ready for predict()


...

Ready for predict()


2024-07-04 02:30:22.406475: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-04 02:30:23.289269: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tu

Ready for predict()
Ready for predict()


Reranking: 100%|██████████| 3/3 [03:43<00:00, 74.37s/it]

3
[0.9456437230110168, 0.996783971786499, 0.9666732549667358, 6.670789844065439e-06, 0.08171119540929794, 0.7397224307060242, 0.008762986399233341, 0.07190988212823868, 2.7369012514100177e-06, 4.171929685981013e-05, 0.03734675794839859, 0.9959157109260559, 7.160047516663326e-06, 0.0015542611945420504, 0.941811740398407, 7.144080427679e-06, 0.0004525045515038073, 0.0003630525025073439, 2.427274921501521e-05, 2.3689328372711316e-05, 0.9712644219398499, 0.24110570549964905, 0.0002644528285600245, 0.5568408966064453, 0.00040787571924738586, 0.9873769879341125, 0.036740224808454514, 0.824500322341919, 0.00019249400065746158, 0.8373112678527832, 0.5780090093612671, 7.149000884965062e-05, 0.9149550199508667, 0.00017964081780519336, 0.9679471850395203, 0.9127671718597412, 0.0001653720100875944, 0.001106139156036079, 1.948814133356791e-05, 5.129407327331137e-06, 0.9169193506240845, 0.058274175971746445, 1.4319725778477732e-06, 0.001757692196406424, 8.745161721890327e-06, 0.0010489268461242318, 

In [ ]:
a = [1,2,3,4,5]
print(a[0:2])

[1, 2]


#### Documents with "same" T5 scores

In [ ]:
from pyserini.search.lucene import LuceneSearcher

retrieval_query_list = [
    "Can you compare mozzarella with plant-based cheese?",
    "No, I mean Academy Awards and Golden Globe Awards."
]
qid_list_string = [
    "10-1-6",
    "15-1-5"
]

query_dict = {
    qid: query for qid, query in zip(qid_list_string, retrieval_query_list)
}

searcher = LuceneSearcher("/part/01/Tmp/yuchen/indexes/clueweb22b_ikat23_fengran_sparse_index_2/")
#searcher = LuceneSearcher("/data/rech/huiyuche/TREC_iKAT_2024/data/indexes/clueweb22b_ikat23_fengran_sparse_index_2")
searcher.set_bm25(0.9, 0.4)
hits = searcher.batch_search(retrieval_query_list, qid_list_string, k = 50, threads = 40)

/data/rech/huiyuche/envs/trec_ikat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Jul 12, 2024 12:05:21 P.M. org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


In [ ]:
import json
def get_doc_content_by_docid(docid, hit_list):
    for doc_object in hit_list:
        if doc_object.docid == docid:
            return doc_object.lucene_document.raw
            #return json.loads(searcher.doc(doc_object.docid).raw())["contents"]
print(get_doc_content_by_docid("clueweb22-en0012-49-03406:9", hits["10-1-6"]))

AttributeError: 'org.apache.lucene.document.Document' object has no attribute 'raw'

In [ ]:
print(get_doc_content_by_docid("clueweb22-en0000-88-08679:9", hits["10-1-6"]))


EXTRA LARGE MARGHERITA $7.00 Diced tomatoes & stretchy mozzarella, topped with oregano. 0 EXTRA LARGE CHEESY GARLIC WITH CRÈME FRAICHE $7.00 Stretchy mozzarella, classic garlic sauce & oregano on a crème fraiche base. 0 MINI PEPPERONI $3.00 A Domino's classic. Slices of crispy pepperoni & creamy mozzarella, made on Deep Pan crust. 0 MINI CHEESY GARLIC WITH CRÈME FRAICHE $3.00 Stretchy mozzarella, classic garlic sauce & oregano, made on Deep Pan crust with crème fraiche. 0 MINI MARGHERITA $3.00 Diced tomatoes & stretchy mozzarella, made on Deep Pan crust & topped with oregano. 0 MINI HAM & CHEESE $3.00 Strips of smokey leg ham & creamy mozzarella, made on Deep Pan crust. 0 MINI SIMPLY CHEESE $3.00 Simply topped with lots of melted mozzarella goodness. Made on Deep Pan crust. 0 VEGETARIAN PLANT-BASED FIRE BREATHER $14.45 Plant-based beef, fiery jalapenos, diced tomato & sliced red onion with a spicy hit of chilli flakes. 0 VEGETARIAN PLANT-BASED GODFATHER $16.65 Plant-based beef, vibrant

In [ ]:

print(get_doc_content_by_docid("clueweb22-en0034-01-09394:9", hits["10-1-6"]))

EXTRA LARGE MARGHERITA $7.00 Diced tomatoes & stretchy mozzarella, topped with oregano. 0 EXTRA LARGE CHEESY GARLIC WITH CRÈME FRAICHE $7.00 Stretchy mozzarella, classic garlic sauce & oregano on a crème fraiche base. 0 MINI PEPPERONI $3.00 A Domino's classic. Slices of crispy pepperoni & creamy mozzarella, made on Deep Pan crust. 0 MINI CHEESY GARLIC WITH CRÈME FRAICHE $3.00 Stretchy mozzarella, classic garlic sauce & oregano, made on Deep Pan crust with crème fraiche. 0 MINI MARGHERITA $3.00 Diced tomatoes & stretchy mozzarella, made on Deep Pan crust & topped with oregano. 0 MINI HAM & CHEESE $3.00 Strips of smokey leg ham & creamy mozzarella, made on Deep Pan crust. 0 MINI SIMPLY CHEESE $3.00 Simply topped with lots of melted mozzarella goodness. Made on Deep Pan crust. 0 VEGETARIAN PLANT-BASED FIRE BREATHER $14.45 Plant-based beef, fiery jalapenos, diced tomato & sliced red onion with a spicy hit of chilli flakes. 0 VEGETARIAN PLANT-BASED GODFATHER $16.65 Plant-based beef, vibrant

# Test Retrieval

#### Repllama

In [ ]:
import sys
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/src/')
from llm import RepllamaDocumentEncoder
import importlib
importlib.reload(sys.modules['llm'])
from llm import RepllamaDocumentEncoder

cache_dir = "/data/rech/huiyuche/huggingface"

/data/rech/huiyuche/envs/trec_ikat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-15 01:43:14.508614: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-15 01:43:14.559720: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 01:43:19.710949: W tensorflow/comp

In [ ]:
import torch
import gc
del repllama_encoder
torch.cuda.empty_cache()
gc.collect()

12823

In [ ]:

repllama_encoder = RepllamaDocumentEncoder(
    cache_dir=cache_dir,
    device_map = "cuda:0",
    quant_4bit = False,
    quant_8bit = False,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.91s/it]


In [ ]:
outputs = repllama_encoder.encode(
    [
       "The capital of France is Paris and Beijing.",
       "Paris is the capital of France." 
        ]
)
print(outputs)

[[-0.01464844 -0.01672363  0.01953125 ...  0.00439453  0.00958252
   0.02478027]
 [ 0.02319336 -0.00154114 -0.00854492 ... -0.01031494  0.00466919
   0.00592041]]


/data/rech/huiyuche/TREC_iKAT_2024/src/llm.py:460: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  passage_embeddings = torch.tensor(passage_embeddings, dtype=torch.float32).numpy()


In [ ]:
import torch
passage_embeddings = outputs.last_hidden_state[:,-1,:].detach().cpu()
passage_embeddings = torch.nn.functional.normalize(passage_embeddings, p=2, dim=1)
print(passage_embeddings.shape)

torch.Size([2, 4096])


In [ ]:
import numpy as np
passage_embeddings = torch.tensor(passage_embeddings, dtype=torch.float32)
print(passage_embeddings.numpy())

[[-0.01452637 -0.01660156  0.01940918 ...  0.0043335   0.00952148
   0.0246582 ]
 [ 0.02331543 -0.00151825 -0.00854492 ... -0.01037598  0.00466919
   0.00595093]]


/tmp/ipykernel_946801/2741188116.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  passage_embeddings = torch.tensor(passage_embeddings, dtype=torch.float32)


In [ ]:
test_tensor = torch.tensor([[1,2,3],[4,5,6],[7,8,9]], dtype=torch.float32)
print(torch.nn.functional.normalize(test_tensor, p=2, dim=1))

tensor([[0.2673, 0.5345, 0.8018],
        [0.4558, 0.5698, 0.6838],
        [0.5026, 0.5744, 0.6462]])


In [ ]:
0.2673**2 + 0.5345**2 + 0.8018**2

1.00002278


repllama_ret

# Test prompter

In [2]:
import sys
import os
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/src/')
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/src/rewrite/')
import importlib
import promptor
import topics
import llm
import rewrite
importlib.reload(sys.modules['promptor'])
importlib.reload(sys.modules['topics'])
importlib.reload(sys.modules['llm'])
importlib.reload(sys.modules['rewrite'])


from promptor import (
    RewriteAndResponsePromptor, 
    PersonalizedCIRQueryExpansionPromptor,
    SummarizePTKBPromptor,
    PersonalizeViaPTKBSummaryPrompter,
    RARPersonalizedCoTPromptor
    )

from llm import OpenAILM, LM

from topics import (
    Turn, 
    load_turns_from_ikat_topic_files, 
    save_turns_to_json, 
    load_turns_from_json,
    Result,
    Reformulation,
    get_context_by_qid,
    get_turn_by_qid
)

from rewrite import(
    get_llm_rm_expansion_terms
)

from dataclasses import asdict
cache_dir = "/data/rech/huiyuche/huggingface"

/data/rech/huiyuche/envs/trec_ikat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-30 13:00:15.758597: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-30 13:00:17.948114: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package words to /u/huiyuche/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /u/huiyuche/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /u/huiyuche/nltk

#### Test Personalized CoT Prompt

In [78]:
turn_list = load_turns_from_json("/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat23/ikat_2023_test.json")
current_turn = get_turn_by_qid("11-1-8",turn_list)

In [82]:
turn_list = load_turns_from_json("/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat24/ikat_2024_test.json")
current_turn = get_turn_by_qid("0-3",turn_list)

In [83]:
model_name = "gpt-3.5-turbo"
rewriter = OpenAILM(
        api_key=os.environ['openai_key'],
        model_name= model_name,
        max_tokens=2048,)

# prompter = RARPersonalizedCoTPromptor(
#     demo_file = "/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat24/demonstration_using_ikat24.json",
#     enable_cot=True,
#     zero_shot_cot=False,
#     one_shot_cot=True,
#     cot_format="cot_seperate"
#     )

prompter = RARPersonalizedCoTPromptor(
    demo_file = "/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat23/demonstration_using_ikat23.json",
    enable_cot=True,
    zero_shot_cot=False,
    one_shot_cot=True,
    cot_format="cot_seperate"
    )
# prompter = RARPersonalizedCoTPromptor(
#     demo_file = "/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat24/demonstration_using_ikat24.json",
#     enable_cot=True,
#     zero_shot_cot=True,
#     one_shot_cot=False,
#     )

context = get_context_by_qid(current_turn.turn_id,turn_list)
current_turn_ptkb_dict = current_turn.ptkb
prompt = prompter.build_turn_prompt(context,current_turn_ptkb_dict,current_turn)
print(prompt)

# Task Description:
You will be given
	(1) An information-seeking dialog between an user and an intelligent assistant.
	(2) The profile of the user, in form of several sentences describing his/her background information.
Your tasks are as follows:
	(1) Help the assistant rewrite the user's question such that:
		a. The rewritten question can fully express the user's information needs without the need of dialog context. 
		b. The assistant could use the rewritten question as a search engine query to gather supporting documents that can help answer the user's question.
		c. Please analyze the quesiton's nature, and judge if it is necessary to personalize. If so, please add personalized elements to the question based on the user's profile.
	(2) Provide your reasoning process in terms of how to adopt de-contextualizaiton (a. & b.) as well as personalization (c.) before rewriting the question. 
	(3) Provide an informative response to the question.

Now, I will give you several example multi-

In [13]:
response = rewriter.generate_text(prompt)
print(response)
liste = prompter.parse_returned_text(response[0])
print(liste)

['Reason: De-contextualization: The user is inquiring about the DASH diet, which is a specific diet plan. Personalization: Since the user profile mentions specific dietary restrictions and health conditions, it is important to consider these factors when providing information about the DASH diet.\nSo the question should be rewritten as:\nRewrite: Can you provide information about the DASH diet considering my vegetarian, lactose intolerant, and soybean allergy conditions?\nResponse: The DASH (Dietary Approaches to Stop Hypertension) diet is a dietary plan that emphasizes fruits, vegetables, whole grains, and lean proteins. It is low in saturated fats, cholesterol, and sodium, which can be beneficial for individuals with heart problems like you. The DASH diet can be adapted to vegetarian preferences by focusing on plant-based protein sources and dairy alternatives for calcium. However, since you are lactose intolerant and allergic to soybeans, you may need to make further modifications t

#### test ptkb summarize + decontextulized query

In [46]:
model_name = "gpt-3.5-turbo"
rewriter = OpenAILM(
        api_key=os.environ['openai_key'],
        model_name= model_name)
prompter= PersonalizeViaPTKBSummaryPrompter(
    enable_cot = False,
)
prompter_cot= PersonalizeViaPTKBSummaryPrompter(
    enable_cot = True,
)

In [35]:
turn_list = load_turns_from_json("/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat23/ikat_2023_test.json")
current_turn = get_turn_by_qid("9-1-3",turn_list)
ptkb_summary = current_turn.find_reformulation("ptkb_summarize").reformulated_query
decontextualized_query = current_turn.find_reformulation("rar_rw").reformulated_query

In [37]:
prompt = prompter.build_turn_prompt(
    summary = ptkb_summary,
    user_query = decontextualized_query,
)
prompt_cot = prompter_cot.build_turn_prompt(
    summary = ptkb_summary,
    user_query = decontextualized_query,
)
print(prompt)
#print(prompt_cot)

You will receive a **Profile Summary** of a search engine user, as well as the **User's Question** presented to the search engine. Your task is to:
	1. Reformulate the query by incorporating relevant information from the user's profile to create a personalized query.
	2. Provide a personalized response to the reformulated query, ensuring it is tailored to the user's specific needs and context.

Here is the user's **Profile Summary**:
The user is a vegetarian, lactose intolerant, and allergic to soybeans. They have kidney and heart problems, which affect their diet and exercise routine. They recently finished watching Game of Thrones and were disappointed with the ending. They are considering changing their phone and are an Android user.

Here is the **User's Question**:
What are the key aspects of the DASH diet, and how does it compare to other diets for weight loss?

Now, please provide the personalized reformulated user question and the personalized response. The output format should

In [47]:
response = rewriter.generate_text(prompt)
print(response)
liste = prompter.parse_returned_text(response[0])
print(liste)

['Rewrite: What are the key aspects of the DASH diet that would be suitable for a vegetarian, lactose intolerant, and soybean allergic individual with kidney and heart problems, and how does it compare to other diets for weight loss?\nResponse: The DASH diet focuses on fruits, vegetables, whole grains, and lean proteins, making it suitable for a vegetarian with dietary restrictions. It is low in sodium, which is beneficial for individuals with kidney and heart problems. Compared to other diets, the DASH diet has been shown to be effective for weight loss while also promoting overall health and managing conditions like hypertension. It may be a good option for you given your specific dietary needs and health concerns.']
['What are the key aspects of the DASH diet that would be suitable for a vegetarian, lactose intolerant, and soybean allergic individual with kidney and heart problems, and how does it compare to other diets for weight loss?', 'The DASH diet focuses on fruits, vegetables

In [48]:
response = rewriter.generate_text(prompt_cot)
print(response)
liste = prompter.parse_returned_text(response[0])
print(liste)

['Reason: The user has kidney and heart problems, which affect their diet and exercise routine. They are also a vegetarian, lactose intolerant, and allergic to soybeans.\nRewrite: What are the key aspects of the DASH diet for someone with kidney and heart problems, who is a vegetarian, lactose intolerant, and allergic to soybeans? How does the DASH diet compare to other diets for weight loss in this specific context?\nResponse: The DASH diet, which stands for Dietary Approaches to Stop Hypertension, focuses on reducing sodium intake and increasing consumption of fruits, vegetables, whole grains, and lean proteins. For someone with kidney and heart problems, being a vegetarian, lactose intolerant, and allergic to soybeans, the DASH diet can be adapted by incorporating plant-based proteins like legumes, nuts, and seeds, and choosing lactose-free dairy alternatives. It is important to consult with a healthcare provider or a dietitian to tailor the DASH diet to your specific dietary restri

#### test ptkb summarize prompter

In [10]:
model_name = "gpt-4o-2024-08-06"
rewriter = OpenAILM(
        api_key=os.environ['openai_key'],
        model_name= model_name,
        max_tokens=2048,)
prompter= SummarizePTKBPromptor()


In [4]:
turn_list = load_turns_from_json("/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat23/ikat_2023_test.json")
context = get_context_by_qid( "9-1-3",turn_list)
current_turn = get_turn_by_qid("9-1-3",turn_list )

In [12]:
prompt = prompter.build_turn_prompt(current_turn.ptkb)
print(prompt)

You will be given a persona of a search engine user, in form of several sentences describing his/her background information. Please provide a summary of the persona. The summary should cover all the key points and main ideas presented in the original persona, while also condensing the information into a concise and easy-to-understand format. The length of the summary should be appropriate to capture the main points and key details of the text, without including unnecessary information or becoming overly long.

Here is the **User Persona**:

1. I don't like the new spin-off; because I keep comparing the two and it has lower quality.
2. Because of my kidney problem, I have to drink water frequently to stay hydrated.
3. I'm going to change my phone.
4. I can't exercise too much because of the heart problem that I have.
5. I'm vegetarian.
6. I'm lactose intolerant.
7. I'm allergic to soybeans.
8. I just finished watching the Game of Thrones.
9. I didn't like how the series ended, especiall

In [13]:
response = rewriter.generate_text(prompt)
print(response)

["Summary: The user is a vegetarian, lactose intolerant, and allergic to soybeans. They have kidney and heart problems, which limit their exercise and require frequent water intake. They recently finished watching Game of Thrones and didn't like the ending, especially the war scenes. They are considering changing their Android phone and are not a fan of a new spin-off show they have been watching."]


#### test RAR prompt

In [8]:
prompter = RewriteAndResponsePromptor(
    demo_file = "/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat23/original_demonstration.json",
    enable_cot = False
)
turn_list = load_turns_from_json("/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat23/ikat_2023_test.json")
context = get_context_by_qid( "9-1-3",turn_list)
current_turn = get_turn_by_qid("9-1-3",turn_list )

In [47]:
model_name = "gpt-4o-2024-08-06"
rewriter = OpenAILM(
        api_key=os.environ['openai_key'],
        model_name= model_name,
        max_tokens=2048,)
turn_list = load_turns_from_json("/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat24/ikat_2024_test.json")
current_turn = get_turn_by_qid("16-6",turn_list)
context = get_context_by_qid("16-6",turn_list)

In [48]:
prompt = prompter.build_turn_prompt(context,current_turn)
#print(prompt)

In [49]:
response = rewriter.generate_text(prompt)
liste = prompter.parse_returned_text(response[0])


In [50]:
print(response[0])

Rewrite: Hmm, no, alligator hunting is too expensive. Is hog hunting included in the standard hunting license? What about my father, does he need a separate license?

Response: Hog hunting in Florida is generally included in the standard hunting license, and you can hunt hogs year-round on private property without a specific season or bag limit. However, if you plan to hunt on public lands or wildlife management areas, you may need additional permits. As for your father, if he plans to hunt, he will need his own hunting license. If he was born after January 1, 1975, he will also need to complete a hunter education course unless he has already done so in another state that meets the International Hunter Education Association (IHEA-USA) standards.


In [ ]:
Rewrite: Hmm, no, alligator hunting is too expensive. Is hog hunting included in the standard hunting license? What about my father, does he need a separate license?

Response: Hog hunting in Florida is generally included in the standard hunting license, and you can hunt hogs year-round on private property without a specific season or bag limit. However, if you plan to hunt on public lands or wildlife management areas, you may need additional permits. As for your father, if he plans to hunt, he will need his own hunting license. If he was born after January 1, 1975, he will also need to complete a hunter education course unless he has already done so in another state that meets the International Hunter Education Association (IHEA-USA) standards.


In [ ]:
,
            {
                "reformulation_name": "gpt-4o_rar_rw",
                "reformulated_query": "Hmm, no, alligator hunting is too expensive. Is hog hunting included in the standard hunting license? What about my father, does he need a separate license?",
                "ptkb_provenance": []
            },
            {
                "reformulation_name": "gpt-4o_rar_rs",
                "reformulated_query": "Hog hunting in Florida is generally included in the standard hunting license, and you can hunt hogs year-round on private property without a specific season or bag limit. However, if you plan to hunt on public lands or wildlife management areas, you may need additional permits. As for your father, if he plans to hunt, he will need his own hunting license. If he was born after January 1, 1975, he will also need to complete a hunter education course unless he has already done so in another state that meets the International Hunter Education Association (IHEA-USA) standards.",
                "ptkb_provenance": []
            }

In [ ]:
print(prompter.demo)

Example #1:
Question: What should I consider when buying a phone?
Rewrite: What should I consider when buying a phone?
Response: The design of the phone and the overall look and feel of the phone are very important. You should be comfortable with the way the phone looks and feels when you hold it in your hand. In addition, don't be afraid to get a phone from a different manufacturer than you're used to. Consider an older, used, or refurbished phone to save money. Sometimes a year-old or even a two-year-old device might give you everything you need.

Question: Cool. Which one would you recommend?
Rewrite: Cool. Which smartphone would you recommend for me?
Response: Just because a phone has everything and the kitchen sink doesn't mean you have to buy it — it's just an option for those who might have a larger budget. The same logic applies to pretty much anything you can buy. If you want something more affordable, cheaper, but has the latest in Android and that comes with 5000mAh of batte

### test get probabilities

In [2]:
prompt_relevance = prompter_relevance.build_turn_prompt(context,current_turn_ptkb_dict,current_turn)
prompt = prompter.build_turn_prompt(context,current_turn_ptkb_dict,current_turn)

NameError: name 'prompter_relevance' is not defined

#### openAI API

In [ ]:
rewriter = OpenAILM(
    api_key = os.environ['openai_key'],
    model_name = "gpt-3.5-turbo",
    n = 1,
    logprobs = True,
    top_logprobs = 19
    )

In [ ]:

responses = rewriter.generate(prompt)
print(responses)
print(responses.choices[0].logprobs.content[0].top_logprobs)
print(rewriter.generate_text(prompt))

ChatCompletion(id='chatcmpl-9slNyG6w9gq1veAXV4l80ndYqIljU', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Keyword', bytes=[75, 101, 121, 119, 111, 114, 100], logprob=-0.005804013, top_logprobs=[]), ChatCompletionTokenLogprob(token=':', bytes=[58], logprob=-1.9361265e-07, top_logprobs=[]), ChatCompletionTokenLogprob(token=' healthy', bytes=[32, 104, 101, 97, 108, 116, 104, 121], logprob=-0.16378154, top_logprobs=[])]), message=ChatCompletionMessage(content='Keyword: healthy', role='assistant', function_call=None, tool_calls=None))], created=1722838890, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=3, prompt_tokens=700, total_tokens=703))
[]


#### Hugging face

In [4]:
 cache_dir = "/data/rech/huiyuche/huggingface"
 llm_model = LM(
    model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct",
    padding_side="left",
    dtype="bf16",
    device_map= "auto",
    attn_implementation="flash_attention_2",
    access_token=None,
    cache_dir=cache_dir,
    accelerator = None,
    load_in_8bit = False,
    load_in_4bit = False,
)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/data/rech/huiyuche/envs/trec_ikat/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.13it/s]


In [11]:

turn_list = load_turns_from_json("/data/rech/huiyuche/TREC_iKAT_2024/data/topics/ikat23/ikat_2023_test.json")
current_turn = get_turn_by_qid("9-1-3",turn_list)

# P -> personalize, D -> demo, C -> cot, Re -> rel explain
# O -> oracle, Rf -> rel feedback
# "raw_llm_rm_PDCReORf"

choices_1 = "raw_llm_rm_P__Re___"
choices_2 = "raw_llm_rm____Re___"
expansion_term_dict_1,prompt_1 = get_llm_rm_expansion_terms(
    prompter_choices = "P__Re___",
    turn = current_turn,
    turn_list = turn_list,
    llm_model = llm_model,
    num_expansion_terms = 100,
)
expansion_term_dict_2, prompt_2 = get_llm_rm_expansion_terms(
    prompter_choices = "___Re___",
    turn = current_turn,
    turn_list = turn_list,
    llm_model = llm_model,
    num_expansion_terms = 100,
)

expansion_term_dict_3, prompt_3 = get_llm_rm_expansion_terms(
    prompter_choices = "________",
    turn = current_turn,
    turn_list = turn_list,
    llm_model = llm_model,
    num_expansion_terms = 100,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [13]:
print(prompt_1)

You will be given an information-seeking dialog between an user and a system, as well as the persona of the user. Your tasks are as follows:

	1. Infer the user's underlying information need expressed by the last question, with the aid of the provided dialog and user persona.
	2. Suggest an expansion term to add in the last question, which corresponds to the keyword that has the highest probability to appear in the set of all possible relevant documents. Please suggest based on your understanding of the user's personalized information need and what should a relevant document contain.

Please note that in this context, 'relevance' refers to documents that contain information related to the user's information need. This is distinct from authenticity, as a relevant document does not necessarily have to be accurate or correct. The key criterion is that the document's topic should align with the user's search intent. For example, both 'The capital of France is Beijing' and 'The capital of F

In [6]:
print(expansion_term_dict_1)

{'hypertension': 17.75, 'heart': 16.5, 'blood': 15.3125, 'Hyp': 14.8125, 'sodium': 14.75, 'low': 14.75, 'hypert': 14.375, 'Heart': 14.1875, 'potassium': 13.625, 'vegetarian': 13.5625, 'Blood': 13.5625, 'healthy': 13.25, 'Veget': 13.125, 'high': 13.0, 'Sodium': 12.875, 'Low': 12.6875, 'syst': 12.625, 'Mediterranean': 12.5, 'hyp': 12.5, 'High': 12.375, '"': 12.125, 'non': 12.0, 'cardiovascular': 12.0, 'restrictive': 11.9375, 'Healthy': 11.9375, 'nutrition': 11.875, 'sugar': 11.8125, 'restrictions': 11.6875, 'balanced': 11.5, 'salt': 11.0625, 'diabetes': 10.9375, 'sustainable': 10.9375, 'cardiac': 10.8125, 'Lifestyle': 10.75, 'lifestyle': 10.75, 'vegetables': 10.625, 'vegan': 10.5, 'S': 10.4375, 'restricted': 10.4375, 'limitations': 10.4375, 'Non': 10.375, 'electroly': 10.375, 'plant': 10.3125, 'diet': 10.25, 'Salt': 10.25, 'restriction': 10.25, 'vegetable': 10.1875, 'benefits': 10.1875, 'syndrome': 10.125, 'components': 10.125, 'kidney': 10.0625, 'cardi': 10.0625, 'restrict': 10.0, 'stro

In [7]:
print(expansion_term_dict_2)

{'hypertension': 16.75, 'healthy': 16.125, 'heart': 15.9375, 'blood': 14.6875, 'Healthy': 14.5625, 'nutrition': 14.5, 'Heart': 14.0625, 'Hyp': 14.0, 'sodium': 13.75, 'Blood': 13.75, 'low': 13.4375, 'Mediterranean': 13.0625, 'benefits': 12.9375, 'hypert': 12.75, 'weight': 12.75, 'Nutrition': 12.75, '"': 12.75, 'syst': 12.625, 'diet': 12.5625, 'health': 12.5, 'Sodium': 12.4375, 'Weight': 12.3125, 'Nut': 12.25, 'balanced': 12.1875, 'nutrients': 12.0625, 'hyp': 11.9375, 'Health': 11.9375, 'high': 11.875, 'cardiovascular': 11.8125, 'potassium': 11.75, 'High': 11.5625, 'Low': 11.5625, 'S': 11.4375, 'sugar': 11.25, 'nutrient': 11.1875, 'sustainable': 11.1875, 'Dash': 11.125, 'carbohydrates': 11.125, 'Diet': 10.9375, 'restrictive': 10.9375, 'diabetes': 10.8125, 'Benefits': 10.8125, 'salt': 10.75, 'components': 10.6875, 'guidelines': 10.5625, 'Diabetes': 10.5, 'beneficial': 10.4375, 'vegetarian': 10.4375, 'Cardio': 10.4375, 'syndrome': 10.375, 'restrictions': 10.375, 'Veget': 10.375, 'Salt': 10

In [55]:
print(expansion_term_dict_3)

{'hypertension': 16.875, 'heart': 15.1875, 'sodium': 14.9375, 'blood': 14.6875, 'Hyp': 14.5, 'Blood': 14.1875, 'Sodium': 14.0, 'Heart': 14.0, 'healthy': 13.875, 'nutrition': 13.5, 'hypert': 13.125, 'syst': 13.0, 'Mediterranean': 13.0, 'Healthy': 12.875, 'S': 12.875, 'low': 12.625, 'potassium': 12.3125, 'balanced': 12.25, 'Nut': 12.125, 'Nutrition': 12.0625, '"': 11.75, 'nutrients': 11.75, 'restrictive': 11.6875, 'hyp': 11.5625, 'cardiovascular': 11.4375, 'benefits': 11.4375, 'sugar': 11.25, 'syndrome': 11.25, 'diet': 11.25, 'Low': 11.125, 'salt': 11.125, 'High': 11.125, 'restrictions': 11.125, 'nutrient': 11.0, 'high': 11.0, 'components': 10.9375, 'guidelines': 10.875, 'Syndrome': 10.875, 'Salt': 10.875, 'sustainable': 10.8125, 'Dash': 10.6875, 'Veget': 10.625, 'Components': 10.625, 'Guidelines': 10.5625, 'Balanced': 10.5, 'restricted': 10.375, 'carbohydrates': 10.3125, 'vegetarian': 10.25, 'American': 10.25, 'restrict': 10.1875, 'recommendations': 10.1875, 'restriction': 10.1875, 'Car

In [57]:
%%time
import nltk
nltk.download('words')
nltk.download('wordnet')
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()


def is_english_word(word):
    base_form = lemmatizer.lemmatize(word.lower())
    return base_form in words.words()


for word in expansion_term_dict_1.keys():
    print(word, is_english_word(word))

[nltk_data] Downloading package words to /u/huiyuche/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /u/huiyuche/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


hypertension True
sodium True
heart True
blood True
Hyp True
Sodium True
hypert False
potassium True
Blood True
low True
nutrition True
healthy True
Heart True
syst False
Mediterranean True
" False
high True
hyp True
Healthy True
S True
cardiovascular True
High True
Low True
restrictions True
Nut True
Nutrition True
balanced True
nutrients True
salt True
sugar True
restrictive True
benefits True
syndrome True
nutrient True
Dash True
diet True
components True
potatoes True
vegetarian True
sustainable True
Pot True
electroly False
Veget False
Salt True
$ False
diabetes True
lifestyle False
Lifestyle False
restricted True
carbohydrates True
cardiac True
Syndrome True
restriction True
dash True
American False
Components True
guidelines True
Diabetes True
restrict True
limitations True
stroke True
non True
Cardio False
nutritional True
System True
Stroke True
magnesium True


Hy False
system True
vegetables True
nut True
beneficial True
Med False
weight True
combination True
lowering True
Dr False
Sugar True
Combination True
hybrid True
disease True
health True
carbohydrate True
Guidelines True
Diet True
Sprint True
Balanced True
recommendations True
glyc False
hypo True
cardi False
nuts True
Benefits True
protein True
cholesterol True
Di True
Weight True
BP False
Lower True
sulfate False
CPU times: user 2.79 s, sys: 216 ms, total: 3.01 s
Wall time: 3.02 s


: 

In [56]:
print(prompt_1)
print(prompt_2)
print(prompt_3)

You will be given an information-seeking dialog between an user and a system, as well as the persona of the user, in form of several sentences describing the user's background information. Your tasks are as follows:

	1. Infer the user's underlying information need expressed by the last question, with the aid of the provided dialog and user persona.
	2. Suggest an expansion term to add in the last question, which corresponds to the keyword that has the highest probability to appear in the set of all possible relevant documents. Please suggest based on your understanding of the user's information need and what should a relevant document contain.

Please note that in this context, 'relevance' refers to documents that contain information related to the user's information need. This is distinct from authenticity, as a relevant document does not necessarily have to be accurate or correct. The key criterion is that the document's topic should align with the user's search intent. For example,

In [15]:
"-".join(["dsf"])
liste = ["apple","orange","pear","banana","orange"]
deduplication = list(set(liste))
print(deduplication)
# get ['pear', 'banana', 'orange', 'apple']

['pear', 'banana', 'orange', 'apple']


In [1]:
a_dict = {
    "a": 1,
    "b": 2,
    "c": 3
}
print(a_dict.a)

AttributeError: 'dict' object has no attribute 'a'

# Test fusion

In [1]:

import sys
import os
sys.path.append('/data/rech/huiyuche/TREC_iKAT_2024/src/')
import importlib
import fuse
importlib.reload(sys.modules['fuse'])

from fuse import (
    rank_list_fusion
    )

from dataclasses import asdict
cache_dir = "/data/rech/huiyuche/huggingface"

#### test linear weighted score fusion 

In [56]:
# rar + personalized cot
# rank_file0="/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[rar_rw]-S2[none]-g[raw]-[BM25]-[none_4_1_none]-[s2_top50].txt"
# rank_file1="/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[rar_personalized_cot1_rw]-S2[none]-g[none]-[BM25]-[none_4_1_none]-[s2_top50].txt"

# rar + personalized cot both + rs
# rank_file0="/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[rar_rwrs]-S2[none]-g[raw]-[BM25]-[none_4_1_none]-[s2_top50].txt"
# rank_file1="/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[rar_personalized_cot1_rwrs]-S2[none]-g[none]-[BM25]-[none_4_1_none]-[s2_top50].txt"

# rar rs + rar rw
# rank_file0="/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[rar_rw]-S2[none]-g[raw]-[BM25]-[none_4_1_none]-[s2_top50].txt"
# rank_file1="/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[rar_rwrs]-S2[none]-g[raw]-[BM25]-[none_4_1_none]-[s2_top50].txt"

# rar rs + personalized_cot rwrs
rank_file0="/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[rar_rw]-S2[none]-g[raw]-[BM25]-[none_4_1_none]-[s2_top50].txt"
rank_file1="/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[rar_personalized_cot1_rwrs]-S2[none]-g[none]-[BM25]-[none_4_1_none]-[s2_top50].txt"
# rar rw + rar rwrs + personalized cot
# rank_file0="/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[rar_rw_fuse_rar_rwrs]-S2[none]-g[none]-[BM25]-[none_4_1_none]-[s2_top50].txt"
# rank_file1="/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[rar_personalized_cot1_rw]-S2[none]-g[none]-[BM25]-[none_4_1_none]-[s2_top50].txt"
topk=1000 
alpha=0.2   # 0.5 for cot no rs, 0.8 for both rs,
fuse_who = "rar_rw_fuse_rar_personalized_cot1_rwrs"
run_name='linear_weighted_score_fusion'

output=f"/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[{fuse_who}]-S2[none]-g[none]-[BM25]-[none_4_1_none]-[s2_top50].txt"

print(output)

/data/rech/huiyuche/TREC_iKAT_2024/results/ClueWeb_ikat/ikat_23_test/ranking/S1[rar_rw_fuse_rar_personalized_cot1_rwrs]-S2[none]-g[none]-[BM25]-[none_4_1_none]-[s2_top50].txt


In [57]:
rank_list_fusion(
    rank_file0,
    rank_file1,
    topk,
    output,
    alpha,
    run_name,
)

Read ranked list0...
Read ranked list1...


Fusing 176 queries (0.002 s/query)
Finished.
